# Mod 3 Final Project

Jeremy Owens, Georgina Sampson

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from api_keys import jo_api_key, gs_api_key
import matplotlib.pyplot as plt
import requests
import re
from bs4 import BeautifulSoup
import time
import copy
from collections import Counter
%matplotlib inline

In [47]:
crime_data = pd.read_csv('denver_data/crime_short.csv')
dispensary_data = pd.read_csv('denver_data/marijuana_active_business_licenses.csv')

In [48]:
print(len(crime_data))
crime_data.head()

450000


,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [49]:
crime_data.columns = [x.lower() for x in crime_data.columns]

In [50]:
crime_data = crime_data[crime_data.is_crime == 1]

In [51]:
crime_data['year'] = crime_data.reported_date.apply(lambda date: int(date.split()[0][-4:]))

In [52]:
offense_type_ids = list(crime_data.groupby('offense_type_id').count().index)

In [53]:
violent_list = [off for off in offense_type_ids if 'assault' in off] + \
                    [off for off in offense_type_ids if 'homicide' in off] + \
                    [off for off in offense_type_ids if 'robbery' in off] + \
                    [off for off in offense_type_ids if 'sex-aslt' in off] + \
                    ['agg-aslt-police-weapon', 'aslt-agg-police-gun']
violent_list.pop(violent_list.index('traf-vehicular-assault'))

drug_list = [off for off in offense_type_ids if 'drug' in off]

In [54]:
def gen_offense_type(offense):
    if offense in violent_list:
        return 'violent'
    elif offense in drug_list:
        return 'drug'
    else:
        return 'other'

In [55]:
crime_data['gen_offense_type'] = crime_data.offense_type_id.apply(lambda offense: gen_offense_type(offense))

In [56]:
crime_data_short = crime_data.drop(['offense_code', 'offense_id', 'offense_code_extension', 'offense_type_id', 
                                    'offense_category_id', 'first_occurrence_date', 'last_occurrence_date', 
                                    'incident_address', 'geo_x', 'geo_y', 'geo_lon', 'geo_lat', 'district_id', 
                                    'precinct_id', 'is_crime', 'is_traffic', 'reported_date'], 
                                    axis=1)
crime_data_short.sample(10)

,incident_id,neighborhood_id,year,gen_offense_type
315614,20156002680,cheesman-park,2015,other
390937,2017458133,cbd,2017,drug
66425,2014315241,five-points,2014,other
355376,2017340455,villa-park,2017,other
212979,20186002238,west-colfax,2018,other
346547,2017320932,five-points,2017,other
45868,2019343093,stapleton,2019,violent
227646,2017414156,capitol-hill,2017,other
74417,2014256296,west-colfax,2014,other
236600,2019239659,cherry-creek,2019,other


In [75]:
crime_data_grouped = crime_data_short.groupby(['neighborhood_id', 'year', 'gen_offense_type']).count()
crime_data_grouped = crime_data_grouped.reset_index()
crime_data_grouped

,neighborhood_id,year,gen_offense_type,incident_id
0,athmar-park,2014,drug,77
1,athmar-park,2014,other,957
2,athmar-park,2014,violent,91
3,athmar-park,2015,drug,82
4,athmar-park,2015,other,923
5,athmar-park,2015,violent,111
6,athmar-park,2016,drug,72
7,athmar-park,2016,other,998
8,athmar-park,2016,violent,97
9,athmar-park,2017,drug,64


In [13]:
dispensary_data.columns = [x.lower().replace(' ', '_') for x in dispensary_data.columns]
dispensary_data = dispensary_data[['business_file_number', 'license_type', 'facility_zip_code']]

In [26]:
neighborhoods_to_analyze = {}
for key, val in initial_neighborhood_counts.items():
    if val > 1:
        like_neighborhoods = []
        for key2, val2 in zip_neighbor_dict_mod.items():
             if key in val2:
                    like_neighborhoods.append(key2)
        neighborhoods_to_analyze.update({key: like_neighborhoods})
neighborhoods_to_analyze

{'kennedy': [80014, 80111, 80231],
 'fort-logan': [80110, 80123, 80235, 80236],
 'harvey-park-south': [80110, 80236],
 'overland': [80110, 80223],
 'university': [80110, 80210],
 'wellshire': [80110, 80210],
 'cbd': [80202, 80204],
 'five-points': [80202, 80203, 80205, 80216, 80218],
 'highland': [80202, 80211],
 'north-capitol-hill': [80202, 80203, 80205, 80218],
 'union-station': [80202, 80204],
 'capitol-hill': [80203, 80218],
 'civic-center': [80203, 80204],
 'speer': [80203, 80209, 80218],
 'baker': [80204, 80223],
 'barnum': [80204, 80219],
 'barnum-west': [80204, 80219],
 'sloan-lake': [80204, 80211, 80212],
 'valverde': [80204, 80219, 80223],
 'city-park': [80205, 80206],
 'city-park-west': [80205, 80206, 80218],
 'cheesman-park': [80206, 80218],
 'cherry-creek': [80206, 80209],
 'country-club': [80206, 80209, 80218],
 'northeast-park-hill': [80207, 80216],
 'south-park-hill': [80207, 80220],
 'stapleton': [80207, 80216, 80239],
 'washington-park': [80209, 80210],
 'washington-

In [14]:
dispensary_data.dropna(inplace=True)
dispensary_data['zip_code'] = dispensary_data.facility_zip_code.astype('int64')
dispensary_data.drop('facility_zip_code', inplace=True, axis=1)

In [15]:
stores_data = dispensary_data[dispensary_data.license_type.isin(['Retail Marijuana Store', 
                                                                 'Medical Marijuana Center'])]
print(len(stores_data))
stores_data.head(10)

345


,business_file_number,license_type,zip_code
0,2016-BFN-0007402,Retail Marijuana Store,80204
2,2018-BFN-0000461,Retail Marijuana Store,80239
4,2014-BFN-0004811,Retail Marijuana Store,80216
8,2013-BFN-1069615,Retail Marijuana Store,80210
9,2013-BFN-1069055,Retail Marijuana Store,80202
11,2016-BFN-0004809,Retail Marijuana Store,80223
12,2015-BFN-0002256,Medical Marijuana Center,80211
19,2018-BFN-0007446,Retail Marijuana Store,80205
20,2010-BFN-1045700,Medical Marijuana Center,80209
21,2010-BFN-1048389,Medical Marijuana Center,80216


In [16]:
store_count = stores_data.groupby('zip_code')['business_file_number'].count()
store_count

zip_code
80123     2
80127     2
80202    13
80203    18
80204    25
80205    22
80206     5
80207     6
80209     8
80210    35
80211    20
80212     6
80214     2
80216    45
80218     6
80219    21
80220    15
80222     9
80223    39
80224    10
80231     4
80232     4
80236     2
80237     2
80239    20
80247     2
80249     2
Name: business_file_number, dtype: int64

In [17]:
html_page = requests.get('http://www.city-data.com/zipmaps/Denver-Colorado.html')
soup = BeautifulSoup(html_page.content, 'html.parser')

In [18]:
zip_code_list = []
zip_codes = soup.find(class_="zip-codes").findAll('a', recursive=False)
for tag in zip_codes:
    zip_code_list.append(int(tag.text))
zip_code_list[:5]

[80012, 80014, 80110, 80111, 80123]

In [19]:
zip_neighbor_dict = {}
for zip_code in zip_code_list:
    html_page = requests.get('http://www.city-data.com/zips/{}.html'.format(zip_code))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    neighborhoods = []
    for a in soup.find_all('a'):
        if 'neighborhood' in a.text:
            if a.text.split()[-1] == 'neighborhood':
                if len(a.text.split()) < 6:
                    if 'Central Business District' in a.text:
                        neighborhood = 'cbd'
                    elif '(' in a.text:
                        neighborhood = '-'.join(a.text[:(a.text.index('(') - 1)].split()).lower()
                    else:
                        neighborhood = '-'.join(a.text.split()[:-1]).lower()
                    neighborhoods.append(neighborhood)
    zip_neighbor_dict.update({zip_code: neighborhoods})
    time.sleep(0.5)
    
zip_neighbor_dict

{80012: ['aurora-district', 'city-center'],
 80014: ['cherry-creek-district', 'kennedy'],
 80110: ['college-view',
  'fort-logan',
  'harvey-park-south',
  'overland',
  'south-platte',
  'southwest',
  'university',
  'wellshire'],
 80111: ['kennedy'],
 80123: ['fort-logan', 'marston', 'southwest'],
 80202: ['cbd',
  'central-platte-valley',
  'cbd',
  'five-points',
  'highland',
  'lower-downtown',
  'north-capitol-hill',
  'union-station'],
 80203: ['alamo-placita',
  'capitol-hill',
  'civic-center',
  'five-points',
  'golden-triangle',
  'north-capitol-hill',
  'speer',
  'uptown'],
 80204: ['auraria',
  'baker',
  'barnum',
  'barnum-west',
  'cbd',
  'civic-center',
  'colfax',
  'cbd',
  'golden-triangle',
  'lincoln-park',
  'lower-downtown',
  'sheridan-boulevard',
  'sloan-lake',
  'southwest',
  'sun-valley',
  'union-station',
  'valverde',
  'villa-park'],
 80205: ['ballpark',
  'city-park',
  'city-park-west',
  'clayton',
  'cole',
  'curtis-park',
  'five-points',
  

In [20]:
crime_neighborhoods = list(crime_data_short.neighborhood_id.unique())
sorted(crime_neighborhoods)

['athmar-park',
 'auraria',
 'baker',
 'barnum',
 'barnum-west',
 'bear-valley',
 'belcaro',
 'berkeley',
 'capitol-hill',
 'cbd',
 'chaffee-park',
 'cheesman-park',
 'cherry-creek',
 'city-park',
 'city-park-west',
 'civic-center',
 'clayton',
 'cole',
 'college-view-south-platte',
 'congress-park',
 'cory-merrill',
 'country-club',
 'dia',
 'east-colfax',
 'elyria-swansea',
 'five-points',
 'fort-logan',
 'gateway-green-valley-ranch',
 'globeville',
 'goldsmith',
 'hale',
 'hampden',
 'hampden-south',
 'harvey-park',
 'harvey-park-south',
 'highland',
 'hilltop',
 'indian-creek',
 'jefferson-park',
 'kennedy',
 'lincoln-park',
 'lowry-field',
 'mar-lee',
 'marston',
 'montbello',
 'montclair',
 'north-capitol-hill',
 'north-park-hill',
 'northeast-park-hill',
 'overland',
 'platt-park',
 'regis',
 'rosedale',
 'ruby-hill',
 'skyland',
 'sloan-lake',
 'south-park-hill',
 'southmoor-park',
 'speer',
 'stapleton',
 'sun-valley',
 'sunnyside',
 'union-station',
 'university',
 'universit

In [21]:
zip_neighbor_dict_mod = copy.deepcopy(zip_neighbor_dict)
zip_neighbor_dict

{80012: ['aurora-district', 'city-center'],
 80014: ['cherry-creek-district', 'kennedy'],
 80110: ['college-view',
  'fort-logan',
  'harvey-park-south',
  'overland',
  'south-platte',
  'southwest',
  'university',
  'wellshire'],
 80111: ['kennedy'],
 80123: ['fort-logan', 'marston', 'southwest'],
 80202: ['cbd',
  'central-platte-valley',
  'cbd',
  'five-points',
  'highland',
  'lower-downtown',
  'north-capitol-hill',
  'union-station'],
 80203: ['alamo-placita',
  'capitol-hill',
  'civic-center',
  'five-points',
  'golden-triangle',
  'north-capitol-hill',
  'speer',
  'uptown'],
 80204: ['auraria',
  'baker',
  'barnum',
  'barnum-west',
  'cbd',
  'civic-center',
  'colfax',
  'cbd',
  'golden-triangle',
  'lincoln-park',
  'lower-downtown',
  'sheridan-boulevard',
  'sloan-lake',
  'southwest',
  'sun-valley',
  'union-station',
  'valverde',
  'villa-park'],
 80205: ['ballpark',
  'city-park',
  'city-park-west',
  'clayton',
  'cole',
  'curtis-park',
  'five-points',
  

In [63]:
for key, values in zip_neighbor_dict_mod.items():
    counter = 0
    list_len = len(values)
    if list_len == 0:
        continue
    else:
        while counter < list_len:
            if values[counter] not in crime_neighborhoods:
                del values[counter]
                list_len -= 1
            else:
                counter += 1
for key, values in zip_neighbor_dict_mod.items():
    values = list(set(values))
    zip_neighbor_dict_mod.update({key: values})
zip_neighbor_dict_mod

{80014: ['kennedy'],
 80110: ['university',
  'fort-logan',
  'overland',
  'wellshire',
  'harvey-park-south'],
 80111: ['kennedy'],
 80123: ['fort-logan', 'marston'],
 80202: ['union-station',
  'five-points',
  'cbd',
  'highland',
  'north-capitol-hill'],
 80203: ['civic-center',
  'five-points',
  'capitol-hill',
  'speer',
  'north-capitol-hill'],
 80204: ['barnum-west',
  'civic-center',
  'sun-valley',
  'lincoln-park',
  'sloan-lake',
  'union-station',
  'cbd',
  'baker',
  'villa-park',
  'barnum',
  'valverde',
  'auraria'],
 80205: ['cole',
  'five-points',
  'city-park',
  'whittier',
  'clayton',
  'skyland',
  'city-park-west',
  'north-capitol-hill'],
 80206: ['cheesman-park',
  'congress-park',
  'country-club',
  'city-park',
  'city-park-west',
  'cherry-creek'],
 80207: ['south-park-hill',
  'northeast-park-hill',
  'north-park-hill',
  'stapleton'],
 80209: ['washington-park-west',
  'belcaro',
  'washington-park',
  'country-club',
  'speer',
  'cherry-creek'],
 

In [25]:
initial_neighborhood_counts = Counter([value for sublist in zip_neighbor_dict_mod.values() for value in sublist])
initial_neighborhood_counts

Counter({'kennedy': 3,
         'fort-logan': 4,
         'harvey-park-south': 2,
         'overland': 2,
         'university': 2,
         'wellshire': 2,
         'marston': 1,
         'cbd': 4,
         'five-points': 5,
         'highland': 2,
         'north-capitol-hill': 4,
         'union-station': 2,
         'capitol-hill': 2,
         'civic-center': 2,
         'speer': 3,
         'auraria': 1,
         'baker': 2,
         'barnum': 2,
         'barnum-west': 2,
         'lincoln-park': 1,
         'sloan-lake': 3,
         'sun-valley': 1,
         'valverde': 3,
         'villa-park': 1,
         'city-park': 2,
         'city-park-west': 3,
         'clayton': 1,
         'cole': 1,
         'skyland': 1,
         'whittier': 1,
         'cheesman-park': 2,
         'cherry-creek': 2,
         'congress-park': 1,
         'country-club': 3,
         'north-park-hill': 1,
         'northeast-park-hill': 2,
         'south-park-hill': 2,
         'stapleton': 3,
      

In [121]:
neighborhoods_df = pd.DataFrame.from_dict(zip_neighbor_dict_mod, orient='index')
neighborhoods_df.dropna(how='all', inplace=True)
neighborhoods_df = neighborhoods_df.reset_index()
neighborhoods_df.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11
0,80014,kennedy,None,None,None,None,None,None,None,None,None,None,None
1,80110,university,fort-logan,overland,wellshire,harvey-park-south,None,None,None,None,None,None,None
2,80111,kennedy,None,None,None,None,None,None,None,None,None,None,None
3,80123,fort-logan,marston,None,None,None,None,None,None,None,None,None,None
4,80202,union-station,five-points,cbd,highland,north-capitol-hill,None,None,None,None,None,None,None


In [122]:
def calc_crime(zip_code, year, crime_type):
    sum_of_crime = 0
    if crime_type == 'violent':
        for neighborhood in zip_neighbor_dict_mod[zip_code]:
            sum_of_crime += crime_data_grouped[(crime_data_grouped.neighborhood_id == neighborhood) & 
                                               (crime_data_grouped.year == year) &
                                               (crime_data_grouped.gen_offense_type == 'violent')]['incident_id'].sum()
    elif crime_type == 'drug':
        for neighborhood in zip_neighbor_dict_mod[zip_code]:
            sum_of_crime += crime_data_grouped[(crime_data_grouped.neighborhood_id == neighborhood) & 
                                               (crime_data_grouped.year == year) &
                                               (crime_data_grouped.gen_offense_type == 'drug')]['incident_id'].sum()
        
    elif crime_type == 'all':
        for neighborhood in zip_neighbor_dict_mod[zip_code]:
            sum_of_crime += crime_data_grouped[(crime_data_grouped.neighborhood_id == neighborhood) & 
                                               (crime_data_grouped.year == year)]['incident_id'].sum()
            
    return int(round(sum_of_crime / len(zip_neighbor_dict_mod[zip_code]), 0))


In [123]:
neighborhoods_df['2014_all_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2014, 'all'))
neighborhoods_df['2014_violent_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2014, 'violent'))
neighborhoods_df['2014_drug_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2014, 'drug'))

neighborhoods_df['2017_all_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2017, 'all'))
neighborhoods_df['2017_violent_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2017, 'violent'))
neighborhoods_df['2017_drug_crime'] = neighborhoods_df['index'].apply(lambda index: calc_crime(index, 2017, 'drug'))


In [124]:
neighborhoods_df = neighborhoods_df.drop(range(12), axis=1)
neighborhoods_df.rename({'index': 'zip_code'}, axis=1, inplace=True)
neighborhoods_df.head()

,zip_code,2014_all_crime,2014_violent_crime,2014_drug_crime,2017_all_crime,2017_violent_crime,2017_drug_crime
0,80014,250,41,3,335,51,9
1,80110,389,45,19,388,38,20
2,80111,250,41,3,335,51,9
3,80123,296,30,4,417,53,4
4,80202,2012,246,205,2078,275,181
5,80203,1721,232,156,2085,273,250
6,80204,1102,130,92,1086,138,81
7,80205,930,122,80,1024,141,91
8,80206,550,53,17,555,57,15
9,80207,993,88,40,1036,97,46


In [138]:
final_df = neighborhoods_df.join(store_count, on='zip_code')
final_df.fillna(0, inplace=True)
final_df.rename({'business_file_number': 'dispensary_count'}, axis=1, inplace=True)
final_df = final_df.astype({'dispensary_count': 'int64'})
final_df

,zip_code,2014_all_crime,2014_violent_crime,2014_drug_crime,2017_all_crime,2017_violent_crime,2017_drug_crime,dispensary_count
0,80014,250,41,3,335,51,9,0
1,80110,389,45,19,388,38,20,0
2,80111,250,41,3,335,51,9,0
3,80123,296,30,4,417,53,4,2
4,80202,2012,246,205,2078,275,181,13
5,80203,1721,232,156,2085,273,250,18
6,80204,1102,130,92,1086,138,81,25
7,80205,930,122,80,1024,141,91,22
8,80206,550,53,17,555,57,15,5
9,80207,993,88,40,1036,97,46,6


# Working Zone

In [ ]:
print(len(crime_data))
crime_data.isna().sum()

In [ ]:
print(len(crime_data))
crime_data.sample(10)

In [ ]:
html_page = requests.get('http://www.city-data.com/zips/{}.html'.format(80202))
soup = BeautifulSoup(html_page.content, 'html.parser')
neighborhoods = []
for a in soup.find_all('a'):
    if 'neighborhood' in a.text:
        if a.text.split()[-1] == 'neighborhood':
            if len(a.text.split()) < 6:
                if 'Central Business District' in a.text:
                    neighborhood = 'cbd'
                elif '(' in a.text:
                    neighborhood = '-'.join(a.text[:(a.text.index('(') - 1)].split()).lower()
                else:
                    neighborhood = '-'.join(a.text.split()[:-1]).lower()
                neighborhoods.append(neighborhood)
neighborhoods

In [26]:
neighborhoods_to_analyze = {}
for key, val in initial_neighborhood_counts.items():
    if val > 1:
        like_neighborhoods = []
        for key2, val2 in zip_neighbor_dict_mod.items():
             if key in val2:
                    like_neighborhoods.append(key2)
        neighborhoods_to_analyze.update({key: like_neighborhoods})
neighborhoods_to_analyze

{'kennedy': [80014, 80111, 80231],
 'fort-logan': [80110, 80123, 80235, 80236],
 'harvey-park-south': [80110, 80236],
 'overland': [80110, 80223],
 'university': [80110, 80210],
 'wellshire': [80110, 80210],
 'cbd': [80202, 80204],
 'five-points': [80202, 80203, 80205, 80216, 80218],
 'highland': [80202, 80211],
 'north-capitol-hill': [80202, 80203, 80205, 80218],
 'union-station': [80202, 80204],
 'capitol-hill': [80203, 80218],
 'civic-center': [80203, 80204],
 'speer': [80203, 80209, 80218],
 'baker': [80204, 80223],
 'barnum': [80204, 80219],
 'barnum-west': [80204, 80219],
 'sloan-lake': [80204, 80211, 80212],
 'valverde': [80204, 80219, 80223],
 'city-park': [80205, 80206],
 'city-park-west': [80205, 80206, 80218],
 'cheesman-park': [80206, 80218],
 'cherry-creek': [80206, 80209],
 'country-club': [80206, 80209, 80218],
 'northeast-park-hill': [80207, 80216],
 'south-park-hill': [80207, 80220],
 'stapleton': [80207, 80216, 80239],
 'washington-park': [80209, 80210],
 'washington-

In [ ]:
violent_data = crime_data[crime_data['gen_offense_type'] == 'violent']
drug_data = crime_data[crime_data['gen_offense_type'] == 'drug']

In [ ]:
offense_data.head(10)

In [ ]:
offense_data.groupby('OFFENSE_CATEGORY_ID')['OFFENSE_CODE'].count()

In [ ]:
print(len(dispensary_data))
dispensary_data.sample(10)

In [ ]:
dispensary_data.groupby('Trade Name').count()

In [ ]:
dispensary_data[dispensary_data['Trade Name'] == '1136 YUMA']